In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # to print multiple outputs from the same cell
import math
import utils
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from operator import index
from collections import defaultdict
from scipy.stats import pearsonr
from datetime import datetime
#from lingua import Language, LanguageDetectorBuilder

Creates the combined dataset of the cleaned and generalized user and tweet datasets. The file that is created is the merged_dataset_cleaned

In [ ]:
tweets_df = pd.read_csv("dataset/tweets_dataset_cleaned.csv")
users_df = pd.read_csv("dataset/users_dataset_cleaned.csv")

In [ ]:
merged_df = pd.merge(users_df, tweets_df, left_on=["user_id"], right_on=["user_id"],)

In [ ]:
merged_df.drop(columns="user_id", inplace=True)
merged_df.rename(columns={"id_x" : "user_id"}, inplace=True)
merged_df.rename(columns={"id_y" : "tweet_id"}, inplace=True)
merged_df.rename(columns={"created_at_x" : "account_created"}, inplace=True)
merged_df.rename(columns={"created_at_y" : "tweet_created"}, inplace=True)
merged_df.head(1)

merged_df.to_csv("./dataset/merged_dataset_cleaned.csv",index=False) # Removes the counting of the Index rows
#works well but problems with wrong values (like float)